#**DSP AT2 Centralized Code Repositories + Github Usage Analysis + Reflection**



## **Preprocessing**

### **Load Packages**

In [ ]:
# Data packages
import pandas as pd      
import numpy as np  
import matplotlib.pyplot as plt
import os
import psycopg2
import sqlalchemy

### **Get raw data**

In [ ]:
# Create data directory
if not os.path.exists('/content/kaggle'):
    os.makedirs('/content/kaggle')

if not os.path.exists('/content/output'):
    os.makedirs('/content/output')
    
# Download data from Kaggle API, unzip and place in data directory
os.environ['KAGGLE_USERNAME'] = "kallikrates"
os.environ['KAGGLE_KEY'] = "98fa396dc991259766a1529a2e4001ac"
!kaggle datasets download -d kallikrates/dsp-at2-bitcoin-dataset
!unzip -q dsp-at2-bitcoin-dataset.zip -d /content/kaggle

### **Process data files for Database loading**

#### **Reshape data and change structure to conform with Database**

**price: Transform file from wide to long, add table identifier dimension, and reorder columns**

In [ ]:
price = pd.read_csv('/content/kaggle/Data_Bitcoin/price.csv', na_filter=False)
price_transformed = price.melt(id_vars=['t'], var_name='Price_Type', value_name='Price')
price_transformed.insert(3, 'Source', 'price')
price_transformed = price_transformed[["t", "Price", "Price_Type","Source"]]

#### **Change structure to conform with Database**

**active-addresses: Add table identifier dimension**

In [ ]:
active_addresses = pd.read_csv('/content/kaggle/Data_Bitcoin/active-addresses.csv', na_filter=False)
active_addresses.insert(2, 'Fact_Type', '')
active_addresses.insert(3, 'Source', 'active_addresses')

**addresses-with-balance-less than or qual to 0-01: Add table identifier dimension**

In [ ]:
addresses_bal_less = pd.read_csv('/content/kaggle/Data_Bitcoin/addresses-with-balance-less than or qual to 0-01.csv', na_filter=False)
addresses_bal_less.insert(2, 'Fact_Type', '')
addresses_bal_less.insert(3, 'Source', 'addresses_bal_less')

**exchange-net-flow-volume: Add table identifier dimension**

In [ ]:
ex_net_flow_vol = pd.read_csv('/content/kaggle/Data_Bitcoin/exchange-net-flow-volume.csv', na_filter=False)
ex_net_flow_vol.insert(2, 'Fact_Type', '')
ex_net_flow_vol.insert(3, 'Source', 'ex_net_flow_vol')

**hash_rate: Add table identifier dimension**

In [ ]:
hash_rate = pd.read_csv('/content/kaggle/Data_Bitcoin/hash-rate.csv', na_filter=False)
hash_rate.insert(2, 'Fact_Type', '')
hash_rate.insert(3, 'Source', 'hash_rate')

**market_cap: Add table identifier dimension**

In [ ]:
market_cap = pd.read_csv('/content/kaggle/Data_Bitcoin/market-cap.csv', na_filter=False)
market_cap.insert(2, 'Fact_Type', '')
market_cap.insert(3, 'Source', 'market_cap')

**new_addresses: Add table identifier dimension**

In [ ]:
new_addresses = pd.read_csv('/content/kaggle/Data_Bitcoin/new-addresses.csv', na_filter=False)
new_addresses.insert(2, 'Fact_Type', '')
new_addresses.insert(3, 'Source', 'new_addresses')

**transaction_count: Add table identifier dimension**

In [ ]:
transaction_count = pd.read_csv('/content/kaggle/Data_Bitcoin/transaction-count.csv', na_filter=False)
transaction_count.insert(2, 'Fact_Type', '')
transaction_count.insert(3, 'Source', 'transaction_count')

**transaction_rate: Add table identifier dimension**

In [ ]:
transaction_rate = pd.read_csv('/content/kaggle/Data_Bitcoin/transaction-rate.csv', na_filter=False)
transaction_rate.insert(2, 'Fact_Type', '')
transaction_rate.insert(3, 'Source', 'transaction_rate')

**transfer_vol_total: Add table identifier dimension**

In [ ]:
transfer_vol_total = pd.read_csv('/content/kaggle/Data_Bitcoin/transfer-volume-total.csv', na_filter=False)
transfer_vol_total.insert(2, 'Fact_Type', '')
transfer_vol_total.insert(3, 'Source', 'transfer_vol_total')

**Rename columns to conform with Database**

In [ ]:
price_transformed.columns = ['Date', 'Fact', 'Fact_Type','Source']
active_addresses.columns = ['Date', 'Fact', 'Fact_Type','Source']
addresses_bal_less.columns = ['Date', 'Fact', 'Fact_Type','Source']
ex_net_flow_vol.columns = ['Date', 'Fact', 'Fact_Type','Source']
hash_rate.columns = ['Date', 'Fact', 'Fact_Type','Source']
market_cap.columns = ['Date', 'Fact', 'Fact_Type','Source']
new_addresses.columns = ['Date', 'Fact', 'Fact_Type','Source']
transaction_count.columns = ['Date', 'Fact', 'Fact_Type','Source']
transaction_rate.columns = ['Date', 'Fact', 'Fact_Type','Source']
transfer_vol_total.columns = ['Date', 'Fact', 'Fact_Type','Source']

**Export all for use in Database**

In [ ]:
price_transformed.to_csv('/content/output/price_transformed.csv', index=False)
active_addresses.to_csv('/content/output/active_addresses.csv', index=False)
addresses_bal_less.to_csv('/content/output/addresses_bal_less.csv', index=False)
ex_net_flow_vol.to_csv('/content/output/ex_net_flow_vol.csv', index=False)
hash_rate.to_csv('/content/output/hash_rate.csv', index=False)
market_cap.to_csv('/content/output/market_cap.csv', index=False)
new_addresses.to_csv('/content/output/new_addresses.csv', index=False)
transaction_count.to_csv('/content/output/transaction_count.csv', index=False)
transaction_rate.to_csv('/content/output/transaction_rate.csv', index=False)
transfer_vol_total.to_csv('/content/output/transfer_vol_total.csv', index=False)

## **Connect to Database**

In [ ]:
# gcloud login and check the DB
!gcloud auth login
!gcloud config set project dsp-at2-group-project
!gcloud sql instances describe dsp-at2-group-postgress-database

In [ ]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
!nohup ./cloud_sql_proxy -instances="dsp-at2-group-project:asia-southeast1:dsp-at2-group-postgress-database"=tcp:5432 &
!sleep 30s

In [ ]:
conn = psycopg2.connect(
    host='127.0.0.1', port='5432', database='bitcoincloud',
    user='postgres', password='Andermatt1')

In [ ]:
qry = "select count(*) from Bitcoin_Fact;"
df = pd.read_sql_query(qry, conn)

# **Query Database**

In [ ]:
qry = "select fact_type, source, count(*) from Bitcoin_Fact group by fact_type, source;"
df = pd.read_sql_query(qry, conn)

In [ ]:
df